<a href="https://colab.research.google.com/github/MinhHeiu0909/thesis_iu_2025/blob/main/finetune_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq, pipeline
from datasets import Dataset, DatasetDict, load_dataset
import torch
import json
import random

In [ ]:
#Load data
dataset_samsum = load_dataset("Heiu/samsum-6k-split")
dataset_samsum


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/597 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/234k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/263k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 4800
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 600
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 600
    })
})

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained("google-t5/t5-base")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def format_with_instruction(example):
    dialogue = example["dialogue"]
    summary = example["summary"]

    input_text = f"""### Instruction:
    Summarize the following dialogue.

    ### Dialogue:
    {dialogue}"""

    return {
        "dialogue": input_text,
        "summary": summary
    }

# 🔹 Apply for the entire dataset
formatted_dataset = dataset_samsum.map(format_with_instruction)

def preprocess(example):
    model_input = tokenizer(example['dialogue'], padding="max_length", truncation=True, max_length=1024)
    target = tokenizer(example['summary'], padding="max_length", truncation=True, max_length=256)
    model_input["labels"] = target["input_ids"]
    return model_input


tokenized_dataset = formatted_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./model-t5",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=7,
    eval_strategy="epoch",
    save_strategy ="epoch",
    eval_steps=250,
    save_steps=250,
    logging_steps=100,
    learning_rate=5e-4,
    warmup_steps=100,
    save_total_limit=7,
    fp16=True,
    report_to="none",
    seed = 9
)



In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-9-2ee1f1e01b7e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train() #resume_from_checkpoint=True

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.194200,0.164581
2,0.156500,0.161702
3,0.137600,0.169654
4,0.122400,0.170679
5,0.117200,0.181702


KeyboardInterrupt: 

In [ ]:
model_path_test = "/content/model-t5/checkpoint-1500"
tokenizer_test = T5TokenizerFast.from_pretrained(model_path_test)
model_test = T5ForConditionalGeneration.from_pretrained(model_path_test)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
for i in range(10):
  input_test = dataset_samsum["test"][i]["dialogue"]
  output_test = dataset_samsum["test"][i]["summary"]
  #6

  # Giả sử bạn đã đưa model sang cuda trước đó
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model_test = model_test.to(device)

  # Tokenize input
  inputs = tokenizer(input_test, return_tensors="pt", padding=True, truncation=True, max_length=1024)
  inputs = {k: v.to(device) for k, v in inputs.items()}  # 🔹 Đưa tất cả tensor sang cùng device với model

  # Generate output
  outputs = model_test.generate(**inputs, max_length=256)

  # Decode
  decoded_output = tokenizer_test.decode(outputs[0], skip_special_tokens=True)

  # Show results
  print(f"  Input     : {input_test}")
  print(f"  Expected  : {output_test}")
  print(f"  Predicted : {decoded_output}")
  print("-" * 80)


  Input     : Jim: Can anyone pick up Alan?
Kate: What happened to the nanny?
Jim: She's done with a flu
Tracy: perfect ;/ I'll pick him up, but he'll need to wait half an hour
Jim: Ok, I'll call the school
  Expected  : Tracy will pick up Alan from school in half an hour.
  Predicted : Tracy will pick up Alan. Tracy's nanny is sick. Jim will call the school and pick up Alan.
--------------------------------------------------------------------------------
  Input     : Hannah: I will be done tm 10:30 so we can meet 11:30
Zuza: yas girl im home
Hannah: ok shouldn't take me long to get there
Zuza: okeeyy
Hannah: and hopefully the paper won't take forever haha
Zuza: oh yes but im worried my best work happens when im drunk XDD
Hannah: soo we getting drunk?? hahah
  Expected  : Hannah will come to Zuza's place tomorrow at 11:30. They will be working on the paper.
  Predicted : Zuza and Hannah will meet at 11:30. Zuza is home. Zuza's best work happens when she's drunk.
----------------------

In [ ]:
sample = """Nina: Did you remember to pick up the dry cleaning?
Tom: Ugh, I totally forgot.
Nina: Tom! We need those clothes for tomorrow’s event.
Tom: I’ll go first thing in the morning, I promise.
Nina: You better. We can’t be late again."""


In [ ]:
# Mount Google Drive nếu chưa mount
from google.colab import drive
drive.mount('/content/drive')

# Đường dẫn lưu mô hình vào Drive
save_path = "/content/drive/MyDrive/Thesis-finetune-model/model-t5-final"

# Lưu model đã fine-tune
model.save_pretrained(save_path)

# Lưu tokenizer kèm theo
tokenizer.save_pretrained(save_path)

print(f"✅ Đã lưu mô hình và tokenizer vào: {save_path}")

Mounted at /content/drive
✅ Đã lưu mô hình và tokenizer vào: /content/drive/MyDrive/Thesis-finetune-model/model-t5-demo


In [ ]:
!zip -r model-t5-900.zip /content/model-t5/checkpoint-900




  adding: content/model-t5/checkpoint-900/ (stored 0%)
  adding: content/model-t5/checkpoint-900/model.safetensors (deflated 8%)
  adding: content/model-t5/checkpoint-900/spiece.model (deflated 48%)
  adding: content/model-t5/checkpoint-900/scaler.pt (deflated 60%)
  adding: content/model-t5/checkpoint-900/rng_state.pth (deflated 25%)
  adding: content/model-t5/checkpoint-900/generation_config.json (deflated 29%)
  adding: content/model-t5/checkpoint-900/optimizer.pt (deflated 8%)
  adding: content/model-t5/checkpoint-900/tokenizer_config.json (deflated 95%)
  adding: content/model-t5/checkpoint-900/config.json (deflated 63%)
  adding: content/model-t5/checkpoint-900/scheduler.pt (deflated 56%)
  adding: content/model-t5/checkpoint-900/tokenizer.json (deflated 74%)
  adding: content/model-t5/checkpoint-900/trainer_state.json (deflated 72%)
  adding: content/model-t5/checkpoint-900/special_tokens_map.json (deflated 86%)
  adding: content/model-t5/checkpoint-900/training_args.bin (deflat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)



Device set to use cuda:0


In [ ]:
sample = """Nina: Did you remember to pick up the dry cleaning?
Tom: Ugh, I totally forgot.
Nina: Tom! We need those clothes for tomorrow’s event.
Tom: I’ll go first thing in the morning, I promise.
Nina: You better. We can’t be late again."""


In [ ]:
print(sample.replace(" .<n>", ".\n"))


Anna: A weird thing just happened to me…
Anna: I ordered an uber, the app said the driver will arrive in three minutes, so I got ready and went down, but the driver was gone. I just received a notification that he ended the journey
Arthur: He probably cancelled it when you didn’t come
Anna: But it said I still have three minutes left. I just the clock and I really wasn’t late. Aren’t they supposed to wait at least 5-10 minutes?
Frank: Yes, they are, I think they should at least call you before driving away
Anna: Exactly! No one called me, I still have time, but he ended the trip before I managed to come down. I live on the first floor for god’s sake
Arthur: Were you charged for it?
Anna: Yeah, the whole sum!
Frank: What?! If he cancelled, you should have paid only a cancellation fee
Anna: That’s why I said it was weird. I think I was simply scammed.
Arthur: Hm, it does seem like that. Have you notified Uber about it? You can send a complaint
Anna: Not yet, but I think I will


In [ ]:
pipe_out = pipe(sample)
print(pipe_out)

Your max_length is set to 200, but your input_length is only 70. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)


[{'summary_text': "Tom forgot to pick up the dry cleaning for tomorrow's event. Nina will go first thing in the morning, so she can't be late again."}]
